# Attack Surface Analysis for Autonomous Vehicles

**Identifying and Mitigating Cybersecurity Risks per ISO/SAE 21434**

Author: Milin Patel  
Institution: Hochschule Kempten - University of Applied Sciences

---

## Learning Objectives

1. Identify attack surfaces in autonomous vehicle systems
2. Analyze attack vectors and threat actors
3. Apply defense-in-depth strategies
4. Implement security monitoring concepts

## 1. Attack Surface Overview

### Autonomous Vehicle Communication Interfaces

```
                    ┌─────────────────┐
                    │   Cloud/OEM     │
                    │   Backend       │
                    └────────┬────────┘
                             │ Cellular/WiFi
    ┌────────────────────────┼────────────────────────┐
    │                   VEHICLE                        │
    │  ┌──────────┐    ┌──────────┐    ┌──────────┐  │
    │  │ Telematics├────┤ Gateway  ├────┤ ADAS ECU │  │
    │  └──────────┘    └────┬─────┘    └──────────┘  │
    │                       │ CAN/Ethernet            │
    │  ┌──────────┐    ┌────┴─────┐    ┌──────────┐  │
    │  │ Infotain.├────┤ Body ECU ├────┤ Sensors  │  │
    │  └──────────┘    └──────────┘    └──────────┘  │
    │       │                                         │
    └───────┼─────────────────────────────────────────┘
            │ Bluetooth/USB/WiFi
    ┌───────┴───────┐    ┌─────────────┐
    │ Mobile Device │    │ V2X (RSU)   │
    └───────────────┘    └─────────────┘
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Set
from enum import Enum
import networkx as nx

print("Attack Surface Analysis Tools Loaded")

## 2. Attack Surface Enumeration

In [ ]:
class AttackVector(Enum):
    PHYSICAL = "Physical Access"
    LOCAL = "Local Network"
    ADJACENT = "Adjacent Network"
    NETWORK = "Remote Network"

class ThreatActor(Enum):
    SCRIPT_KIDDIE = "Script Kiddie"
    HOBBYIST = "Hobbyist/Researcher"
    CRIMINAL = "Organized Criminal"
    NATION_STATE = "Nation State"
    INSIDER = "Malicious Insider"

@dataclass
class AttackSurface:
    """Represents an attack surface in the vehicle system."""
    id: str
    name: str
    interface_type: str
    attack_vector: AttackVector
    connected_assets: List[str]
    protocols: List[str]
    authentication: str
    encryption: str
    known_vulnerabilities: List[str] = field(default_factory=list)
    mitigations: List[str] = field(default_factory=list)

# Define attack surfaces for AV
attack_surfaces = [
    AttackSurface(
        id="AS-001",
        name="OBD-II Port",
        interface_type="Physical",
        attack_vector=AttackVector.PHYSICAL,
        connected_assets=["CAN Bus", "Diagnostics ECU", "Gateway"],
        protocols=["CAN", "UDS", "KWP2000"],
        authentication="None (typically)",
        encryption="None",
        known_vulnerabilities=[
            "Direct CAN bus access",
            "Diagnostic service abuse",
            "Firmware extraction"
        ],
        mitigations=[
            "Physical port protection",
            "Diagnostic authentication (UDS 0x27)",
            "Rate limiting"
        ]
    ),
    AttackSurface(
        id="AS-002",
        name="Telematics Control Unit (TCU)",
        interface_type="Wireless",
        attack_vector=AttackVector.NETWORK,
        connected_assets=["Gateway", "Backend Server", "Infotainment"],
        protocols=["LTE", "5G", "TCP/IP", "TLS"],
        authentication="PKI-based",
        encryption="TLS 1.3",
        known_vulnerabilities=[
            "Cellular protocol attacks",
            "Rogue base station",
            "Backend API vulnerabilities"
        ],
        mitigations=[
            "Mutual TLS authentication",
            "Certificate pinning",
            "Secure boot",
            "Intrusion detection"
        ]
    ),
    AttackSurface(
        id="AS-003",
        name="Infotainment System",
        interface_type="Wireless/Physical",
        attack_vector=AttackVector.ADJACENT,
        connected_assets=["Gateway", "Smartphone", "USB Storage"],
        protocols=["Bluetooth", "WiFi", "USB", "CarPlay/Android Auto"],
        authentication="Bluetooth pairing",
        encryption="BT Secure Simple Pairing",
        known_vulnerabilities=[
            "Bluetooth vulnerabilities (BlueBorne)",
            "USB malware",
            "WiFi attacks",
            "Media file exploits"
        ],
        mitigations=[
            "Network segmentation",
            "Input validation",
            "Sandboxing",
            "Regular updates"
        ]
    ),
    AttackSurface(
        id="AS-004",
        name="V2X Communication",
        interface_type="Wireless",
        attack_vector=AttackVector.ADJACENT,
        connected_assets=["ADAS ECU", "Other Vehicles", "Infrastructure"],
        protocols=["DSRC/802.11p", "C-V2X", "SCMS"],
        authentication="PKI (SCMS)",
        encryption="ECDSA signatures",
        known_vulnerabilities=[
            "Message spoofing",
            "Sybil attacks",
            "Replay attacks",
            "DoS via flooding"
        ],
        mitigations=[
            "SCMS certificate management",
            "Plausibility checks",
            "Misbehavior detection",
            "Rate limiting"
        ]
    ),
    AttackSurface(
        id="AS-005",
        name="Sensor Interfaces (Camera, LiDAR, Radar)",
        interface_type="Physical/Wireless",
        attack_vector=AttackVector.ADJACENT,
        connected_assets=["Perception ECU", "Sensor Fusion Module"],
        protocols=["MIPI CSI-2", "Ethernet", "CAN"],
        authentication="None (physical)",
        encryption="None (internal)",
        known_vulnerabilities=[
            "Adversarial patches",
            "LiDAR spoofing/jamming",
            "Radar interference",
            "GPS spoofing"
        ],
        mitigations=[
            "Sensor fusion cross-validation",
            "Plausibility monitoring",
            "Physical tamper detection",
            "OOD detection"
        ]
    ),
    AttackSurface(
        id="AS-006",
        name="In-Vehicle Network (CAN/Ethernet)",
        interface_type="Wired",
        attack_vector=AttackVector.LOCAL,
        connected_assets=["All ECUs"],
        protocols=["CAN", "CAN-FD", "Automotive Ethernet"],
        authentication="SecOC (optional)",
        encryption="MACsec (optional)",
        known_vulnerabilities=[
            "CAN bus injection",
            "Message replay",
            "DoS via bus flooding",
            "ECU impersonation"
        ],
        mitigations=[
            "SecOC authentication",
            "IDS/IPS",
            "Network segmentation",
            "Message freshness counters"
        ]
    ),
    AttackSurface(
        id="AS-007",
        name="OTA Update Interface",
        interface_type="Wireless",
        attack_vector=AttackVector.NETWORK,
        connected_assets=["All updateable ECUs", "Backend"],
        protocols=["TLS", "Custom OTA protocols"],
        authentication="PKI + Code Signing",
        encryption="AES-256",
        known_vulnerabilities=[
            "Update rollback attacks",
            "Compromised update server",
            "Man-in-the-middle",
            "Incomplete update (brick)"
        ],
        mitigations=[
            "Secure boot chain",
            "Code signing verification",
            "Rollback protection",
            "A/B partition scheme"
        ]
    )
]

print(f"Defined {len(attack_surfaces)} attack surfaces")

In [ ]:
# Display attack surface summary
data = []
for surface in attack_surfaces:
    data.append({
        'ID': surface.id,
        'Name': surface.name,
        'Vector': surface.attack_vector.value,
        'Auth': surface.authentication,
        'Encryption': surface.encryption,
        'Vulns': len(surface.known_vulnerabilities)
    })

df = pd.DataFrame(data)
print("Attack Surface Summary")
print("=" * 80)
print(df.to_string(index=False))

## 3. Attack Tree Analysis

In [ ]:
@dataclass
class AttackNode:
    """Node in attack tree."""
    id: str
    description: str
    node_type: str  # "AND", "OR", "LEAF"
    difficulty: int  # 1-5 (1=easy, 5=very hard)
    resources: int  # 1-5 (1=minimal, 5=extensive)
    children: List['AttackNode'] = field(default_factory=list)
    
    def add_child(self, child: 'AttackNode'):
        self.children.append(child)
    
    @property
    def attack_cost(self) -> float:
        """Calculate attack cost score."""
        return (self.difficulty + self.resources) / 2

def build_perception_attack_tree() -> AttackNode:
    """Build attack tree for perception system compromise."""
    # Root goal
    root = AttackNode(
        id="G0",
        description="Cause perception system to miss critical object",
        node_type="OR",
        difficulty=0,
        resources=0
    )
    
    # Branch 1: Sensor-level attacks
    sensor_attacks = AttackNode(
        id="G1",
        description="Attack sensor inputs",
        node_type="OR",
        difficulty=0,
        resources=0
    )
    
    sensor_attacks.add_child(AttackNode(
        id="A1.1",
        description="Deploy adversarial patch on road/object",
        node_type="LEAF",
        difficulty=2,
        resources=2
    ))
    sensor_attacks.add_child(AttackNode(
        id="A1.2",
        description="LiDAR spoofing with laser",
        node_type="LEAF",
        difficulty=4,
        resources=4
    ))
    sensor_attacks.add_child(AttackNode(
        id="A1.3",
        description="Camera blinding with light source",
        node_type="LEAF",
        difficulty=2,
        resources=1
    ))
    sensor_attacks.add_child(AttackNode(
        id="A1.4",
        description="GPS spoofing",
        node_type="LEAF",
        difficulty=3,
        resources=3
    ))
    
    # Branch 2: Software/Network attacks
    software_attacks = AttackNode(
        id="G2",
        description="Compromise perception software",
        node_type="AND",
        difficulty=0,
        resources=0
    )
    
    network_access = AttackNode(
        id="G2.1",
        description="Gain network access",
        node_type="OR",
        difficulty=0,
        resources=0
    )
    network_access.add_child(AttackNode(
        id="A2.1.1",
        description="Exploit telematics vulnerability",
        node_type="LEAF",
        difficulty=4,
        resources=4
    ))
    network_access.add_child(AttackNode(
        id="A2.1.2",
        description="Compromise infotainment via Bluetooth",
        node_type="LEAF",
        difficulty=3,
        resources=2
    ))
    network_access.add_child(AttackNode(
        id="A2.1.3",
        description="Physical OBD-II access",
        node_type="LEAF",
        difficulty=2,
        resources=2
    ))
    
    software_attacks.add_child(network_access)
    software_attacks.add_child(AttackNode(
        id="A2.2",
        description="Inject malicious CAN messages",
        node_type="LEAF",
        difficulty=3,
        resources=2
    ))
    
    # Branch 3: ML model attacks
    ml_attacks = AttackNode(
        id="G3",
        description="Attack ML model integrity",
        node_type="OR",
        difficulty=0,
        resources=0
    )
    
    ml_attacks.add_child(AttackNode(
        id="A3.1",
        description="Supply chain attack on training data",
        node_type="LEAF",
        difficulty=5,
        resources=5
    ))
    ml_attacks.add_child(AttackNode(
        id="A3.2",
        description="Malicious OTA update of model",
        node_type="LEAF",
        difficulty=5,
        resources=5
    ))
    
    root.add_child(sensor_attacks)
    root.add_child(software_attacks)
    root.add_child(ml_attacks)
    
    return root

def print_attack_tree(node: AttackNode, indent: int = 0):
    """Print attack tree structure."""
    prefix = "  " * indent
    if node.node_type == "LEAF":
        cost_info = f" [D:{node.difficulty}/R:{node.resources}]"
    else:
        cost_info = f" ({node.node_type})"
    print(f"{prefix}{node.id}: {node.description}{cost_info}")
    for child in node.children:
        print_attack_tree(child, indent + 1)

attack_tree = build_perception_attack_tree()
print("Attack Tree: Perception System Compromise")
print("=" * 60)
print("D=Difficulty, R=Resources (1=low, 5=high)")
print()
print_attack_tree(attack_tree)

## 4. Defense-in-Depth Strategy

In [ ]:
# Defense layers for AV
defense_layers = {
    "Layer 1: Perimeter Security": {
        "description": "First line of defense at system boundaries",
        "controls": [
            "Firewall between vehicle domains",
            "TLS for external communications",
            "Certificate-based authentication",
            "Input validation at all interfaces"
        ]
    },
    "Layer 2: Network Security": {
        "description": "In-vehicle network protection",
        "controls": [
            "Network segmentation (safety vs non-safety)",
            "SecOC for CAN message authentication",
            "Intrusion Detection System (IDS)",
            "Rate limiting and anomaly detection"
        ]
    },
    "Layer 3: Host Security": {
        "description": "ECU-level protection",
        "controls": [
            "Secure boot with hardware root of trust",
            "Code signing verification",
            "Memory protection (MPU/MMU)",
            "Secure key storage (HSM)"
        ]
    },
    "Layer 4: Application Security": {
        "description": "Software-level protection",
        "controls": [
            "Secure coding practices",
            "Input validation and sanitization",
            "Privilege separation",
            "Runtime integrity monitoring"
        ]
    },
    "Layer 5: Data Security": {
        "description": "Protection of sensitive data",
        "controls": [
            "Encryption at rest (AES-256)",
            "Encryption in transit (TLS 1.3)",
            "Secure key management",
            "Data integrity verification"
        ]
    },
    "Layer 6: Monitoring & Response": {
        "description": "Detection and incident response",
        "controls": [
            "Security event logging",
            "Real-time threat detection",
            "Incident response procedures",
            "Secure OTA for rapid patching"
        ]
    }
}

print("Defense-in-Depth Strategy")
print("=" * 60)
for layer, details in defense_layers.items():
    print(f"\n{layer}")
    print(f"  {details['description']}")
    print(f"  Controls:")
    for control in details['controls']:
        print(f"    - {control}")

In [ ]:
def visualize_defense_layers():
    """Visualize defense-in-depth layers."""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    layers = list(defense_layers.keys())
    n_layers = len(layers)
    
    # Draw concentric rectangles
    colors = ['#ffcdd2', '#f8bbd9', '#e1bee7', '#c5cae9', '#bbdefb', '#b2dfdb']
    
    for i, (layer, color) in enumerate(zip(reversed(layers), colors)):
        width = 10 - i
        height = 6 - i * 0.5
        rect = plt.Rectangle(
            (i/2, i/4), width, height,
            facecolor=color, edgecolor='black', linewidth=2
        )
        ax.add_patch(rect)
    
    # Add labels
    for i, layer in enumerate(reversed(layers)):
        short_name = layer.split(":")[1].strip()
        ax.text(5, 5.5 - i * 0.8, short_name, ha='center', va='center', 
                fontsize=10, fontweight='bold')
    
    # Add core asset
    ax.text(5, 1.5, "Safety-Critical\nAssets", ha='center', va='center',
            fontsize=12, fontweight='bold', 
            bbox=dict(boxstyle='round', facecolor='yellow', edgecolor='red'))
    
    ax.set_xlim(-0.5, 11)
    ax.set_ylim(-0.5, 7)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Defense-in-Depth: Multiple Security Layers', fontsize=14)
    
    plt.tight_layout()
    plt.show()

visualize_defense_layers()

## 5. Security Controls Mapping

In [ ]:
def map_controls_to_surfaces(surfaces: List[AttackSurface]) -> pd.DataFrame:
    """Map security controls to attack surfaces."""
    all_controls = set()
    for surface in surfaces:
        all_controls.update(surface.mitigations)
    
    # Build control matrix
    data = []
    for control in sorted(all_controls):
        row = {'Control': control}
        for surface in surfaces:
            short_name = surface.name[:15]
            row[short_name] = 'X' if control in surface.mitigations else ''
        data.append(row)
    
    return pd.DataFrame(data)

control_matrix = map_controls_to_surfaces(attack_surfaces)
print("Security Controls Coverage Matrix")
print("=" * 100)
print(control_matrix.to_string(index=False))

## 6. Intrusion Detection Concepts

In [ ]:
class CANMessageAnalyzer:
    """Simple CAN bus intrusion detection system."""
    
    def __init__(self):
        self.message_history = {}
        self.timing_profiles = {}
        self.anomaly_threshold = 3.0  # Standard deviations
    
    def learn_baseline(self, messages: List[Dict]):
        """Learn normal message patterns."""
        for msg in messages:
            can_id = msg['id']
            if can_id not in self.message_history:
                self.message_history[can_id] = []
            self.message_history[can_id].append(msg['timestamp'])
        
        # Calculate timing profiles
        for can_id, timestamps in self.message_history.items():
            if len(timestamps) > 1:
                intervals = np.diff(timestamps)
                self.timing_profiles[can_id] = {
                    'mean': np.mean(intervals),
                    'std': np.std(intervals),
                    'min': np.min(intervals),
                    'max': np.max(intervals)
                }
    
    def detect_anomaly(self, message: Dict) -> Dict:
        """Detect anomalies in incoming message."""
        can_id = message['id']
        timestamp = message['timestamp']
        
        anomalies = []
        
        # Check for unknown CAN ID
        if can_id not in self.timing_profiles:
            anomalies.append({
                'type': 'UNKNOWN_ID',
                'severity': 'HIGH',
                'description': f'Unknown CAN ID: 0x{can_id:03X}'
            })
            return {'anomalies': anomalies, 'is_anomaly': True}
        
        # Check timing
        if can_id in self.message_history and len(self.message_history[can_id]) > 0:
            last_time = self.message_history[can_id][-1]
            interval = timestamp - last_time
            profile = self.timing_profiles[can_id]
            
            z_score = abs(interval - profile['mean']) / (profile['std'] + 1e-6)
            
            if z_score > self.anomaly_threshold:
                anomalies.append({
                    'type': 'TIMING_ANOMALY',
                    'severity': 'MEDIUM',
                    'description': f'Unusual timing for ID 0x{can_id:03X}: {interval:.3f}s (expected {profile["mean"]:.3f}s)',
                    'z_score': z_score
                })
        
        # Update history
        self.message_history[can_id].append(timestamp)
        
        return {
            'anomalies': anomalies,
            'is_anomaly': len(anomalies) > 0
        }

# Simulate CAN traffic and detection
np.random.seed(42)

# Generate normal traffic (baseline)
normal_messages = []
can_ids = [0x100, 0x200, 0x300, 0x400]  # Normal IDs
intervals = [0.01, 0.02, 0.05, 0.1]  # Typical intervals in seconds

t = 0
for _ in range(500):
    for can_id, interval in zip(can_ids, intervals):
        t += interval + np.random.normal(0, interval * 0.05)  # Small jitter
        normal_messages.append({'id': can_id, 'timestamp': t})

# Train detector
detector = CANMessageAnalyzer()
detector.learn_baseline(normal_messages[:400])

print("Learned Timing Profiles")
print("=" * 50)
for can_id, profile in detector.timing_profiles.items():
    print(f"ID 0x{can_id:03X}: mean={profile['mean']*1000:.1f}ms, std={profile['std']*1000:.2f}ms")

In [ ]:
# Test with normal and attack messages
test_messages = [
    {'id': 0x100, 'timestamp': 100.010, 'label': 'Normal'},
    {'id': 0x200, 'timestamp': 100.030, 'label': 'Normal'},
    {'id': 0x500, 'timestamp': 100.031, 'label': 'Injection (Unknown ID)'},
    {'id': 0x100, 'timestamp': 100.032, 'label': 'Injection (Timing)'},
    {'id': 0x300, 'timestamp': 100.080, 'label': 'Normal'},
]

print("\nIntrusion Detection Results")
print("=" * 60)
for msg in test_messages:
    result = detector.detect_anomaly(msg)
    status = "ALERT" if result['is_anomaly'] else "OK"
    print(f"\n[{status}] ID: 0x{msg['id']:03X}, Time: {msg['timestamp']:.3f}s")
    print(f"    Ground Truth: {msg['label']}")
    if result['anomalies']:
        for anomaly in result['anomalies']:
            print(f"    - {anomaly['type']}: {anomaly['description']}")

## 7. Vulnerability Assessment Scoring

In [ ]:
def calculate_cvss_like_score(vulnerability: Dict) -> float:
    """Calculate CVSS-like score for automotive vulnerability."""
    # Base metrics
    av_scores = {'N': 0.85, 'A': 0.62, 'L': 0.55, 'P': 0.20}  # Attack Vector
    ac_scores = {'L': 0.77, 'H': 0.44}  # Attack Complexity
    pr_scores = {'N': 0.85, 'L': 0.62, 'H': 0.27}  # Privileges Required
    ui_scores = {'N': 0.85, 'R': 0.62}  # User Interaction
    
    # Impact metrics
    c_scores = {'H': 0.56, 'L': 0.22, 'N': 0.00}  # Confidentiality
    i_scores = {'H': 0.56, 'L': 0.22, 'N': 0.00}  # Integrity
    a_scores = {'H': 0.56, 'L': 0.22, 'N': 0.00}  # Availability
    s_scores = {'H': 0.56, 'L': 0.22, 'N': 0.00}  # Safety (automotive addition)
    
    # Calculate exploitability
    exploitability = 8.22 * (
        av_scores[vulnerability['attack_vector']] *
        ac_scores[vulnerability['attack_complexity']] *
        pr_scores[vulnerability['privileges_required']] *
        ui_scores[vulnerability['user_interaction']]
    )
    
    # Calculate impact (including safety for automotive)
    impact = 1 - (
        (1 - c_scores[vulnerability['confidentiality']]) *
        (1 - i_scores[vulnerability['integrity']]) *
        (1 - a_scores[vulnerability['availability']]) *
        (1 - s_scores[vulnerability['safety']])
    )
    
    if impact <= 0:
        return 0.0
    
    # Final score
    base_score = min(10, 1.08 * (impact + exploitability))
    return round(base_score, 1)

# Example vulnerabilities
vulnerabilities = [
    {
        'name': 'CAN bus message injection via OBD-II',
        'attack_vector': 'P',  # Physical
        'attack_complexity': 'L',
        'privileges_required': 'N',
        'user_interaction': 'N',
        'confidentiality': 'L',
        'integrity': 'H',
        'availability': 'H',
        'safety': 'H'
    },
    {
        'name': 'Remote code execution via telematics',
        'attack_vector': 'N',  # Network
        'attack_complexity': 'H',
        'privileges_required': 'N',
        'user_interaction': 'N',
        'confidentiality': 'H',
        'integrity': 'H',
        'availability': 'H',
        'safety': 'H'
    },
    {
        'name': 'Bluetooth pairing bypass',
        'attack_vector': 'A',  # Adjacent
        'attack_complexity': 'L',
        'privileges_required': 'N',
        'user_interaction': 'N',
        'confidentiality': 'L',
        'integrity': 'L',
        'availability': 'N',
        'safety': 'N'
    },
    {
        'name': 'Adversarial patch on traffic sign',
        'attack_vector': 'P',
        'attack_complexity': 'L',
        'privileges_required': 'N',
        'user_interaction': 'N',
        'confidentiality': 'N',
        'integrity': 'H',
        'availability': 'N',
        'safety': 'H'
    }
]

print("Vulnerability Scoring (CVSS-like with Safety Impact)")
print("=" * 70)
for vuln in vulnerabilities:
    score = calculate_cvss_like_score(vuln)
    severity = 'Critical' if score >= 9 else 'High' if score >= 7 else 'Medium' if score >= 4 else 'Low'
    print(f"\n{vuln['name']}")
    print(f"  Score: {score:.1f} ({severity})")
    print(f"  Vector: AV:{vuln['attack_vector']}/AC:{vuln['attack_complexity']}/PR:{vuln['privileges_required']}/UI:{vuln['user_interaction']}")
    print(f"  Impact: C:{vuln['confidentiality']}/I:{vuln['integrity']}/A:{vuln['availability']}/S:{vuln['safety']}")

## 8. Key Takeaways

### Attack Surface Management

1. **Enumerate all interfaces**: Document every communication pathway
2. **Prioritize by exposure**: Remote attacks are higher risk than physical
3. **Defense-in-depth**: No single control is sufficient
4. **Continuous monitoring**: Detect and respond to intrusions

### Automotive-Specific Considerations

- Safety and security are interconnected
- Legacy protocols (CAN) lack built-in security
- Long vehicle lifecycles require OTA patching capability
- Sensor attacks are unique to autonomous vehicles

### ISO/SAE 21434 Integration

| Activity | Deliverable |
|----------|-------------|
| Asset identification | Asset catalog |
| Attack surface analysis | Attack surface report |
| Threat modeling | Attack trees, STRIDE analysis |
| Vulnerability assessment | Vulnerability scores, CVSS |
| Control implementation | Defense-in-depth architecture |

## References

1. ISO/SAE 21434:2021 - Road vehicles - Cybersecurity engineering
2. UNECE WP.29 R155 - Cyber security and cyber security management system
3. SAE J3061 - Cybersecurity Guidebook for Cyber-Physical Vehicle Systems
4. NHTSA - Cybersecurity Best Practices for the Safety of Modern Vehicles